In [7]:
import numpy as np

from torch.utils.data import DataLoader
from torchvision import datasets

from tqdm.notebook import tqdm

import MST

In [8]:
BATCH_SIZE = 32
NUM_WORKERS = 4

EPOCHS = 10

In [9]:
transform = np.array

train_dataset = datasets.MNIST(
    root='datasets',
    train=True,
    transform=transform,
    download=True
)

test_dataset = datasets.MNIST(
    root='datasets',
    train=False,
    transform=transform,
    download=True
)

train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
)

test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
)

In [10]:
class MyNet(MST.BasicModule):
    def __init__(self):
        super().__init__()

        self.net_blocks = MST.Sequential(
                MST.Sequential(
                            MST.FullyConnectedLayer(28*28, 100),
                            MST.Sigmoid()
                        ),

                MST.Sequential(
                            MST.FullyConnectedLayer(100, 50),
                            MST.Sigmoid(),
                        ),

                MST.Sequential(
                            MST.FullyConnectedLayer(50, 10),
                            MST.Sigmoid()
                        ),
        )

        # self.fc1 = MST.Sequential(
        #     MST.FullyConnectedLayer(28*28, 100),
        #     MST.Sigmoid()
        # )

        # self.fc2 = MST.Sequential(
        #     MST.FullyConnectedLayer(100, 50),
        #     MST.Sigmoid(),
        # )

        # self.fcRes = MST.Sequential(
        #     MST.FullyConnectedLayer(50, 10),
        #     MST.Sigmoid()
        # )

    def forward(self, x):
        x = x.reshape(-1, 28*28)
        x = self.net_blocks(x)
        # x = self.fc1(x)
        # x = self.fc2(x)
        # x = self.fcRes(x)
        return x

    def backward(self, dOut):
        dN = self.net_blocks.backward(dOut)
        # dN = self.fcRes.backward(dOut)
        # dN = self.fc2.backward(dN)
        # dN = self.fc1.backward(dN)
        return dN

CELoss = MST.CrossEntropyLoss()
net = MyNet()

optimizer = MST.SGD(net, lr=0.01, momentum=0.9)
print(net)

MyNet:
 └── net_blocks: 
	 └── Sequential: 
		 └── FullyConnectedLayer: Trainable(True)
		 └── Sigmoid: Trainable(False)

	 └── Sequential: 
		 └── FullyConnectedLayer: Trainable(True)
		 └── Sigmoid: Trainable(False)

	 └── Sequential: 
		 └── FullyConnectedLayer: Trainable(True)
		 └── Sigmoid: Trainable(False)





In [11]:
def train(net : MST.BasicModule, optimizer : MST.SGD, criterion : MST.BasicModule):
    running_loss = 0
    for images, labels in train_dataloader:
        labels = np.array(labels)
        
        output = net(images)

        loss = criterion(output, labels)
        net.backward(criterion.backward())
        optimizer.step()

        running_loss += loss
    train_loss = running_loss / len(train_dataloader)
    return train_loss


def valid(net : MST.BasicModule, criterion : MST.BasicModule):
    running_loss = 0
    correct_total = 0
    for images, labels in test_dataloader:
        labels = np.array(labels)

        output = net(images)

        loss = criterion(output, labels)
        running_loss += loss

        pred = np.argmax(output, axis=1, keepdims=True)
        correct_total += np.sum(pred[pred==labels.reshape(-1, 1)].astype(bool))
        
    precison = correct_total / len(test_dataloader.dataset)
    valid_loss = running_loss / len(test_dataloader)
    return valid_loss, precison

In [12]:
for epoch in (pbar := tqdm(range(EPOCHS))):
    train_loss = train(net, optimizer, CELoss)
    valid_loss, prec = valid(net, CELoss)

    print(f"[{epoch}] train/valid loss: {train_loss:.4f}/{valid_loss:.4f} prec: {prec:.4f}")
    pbar.set_description(f"train/valid loss: {train_loss:.4f}/{valid_loss:.4f} prec: {prec:.4f}")

  0%|          | 0/10 [00:00<?, ?it/s]

[0] train/valid loss: 4.9759/4.6841 prec: 0.2358
[1] train/valid loss: 4.1218/3.5788 prec: 0.3752
[2] train/valid loss: 3.2781/2.9405 prec: 0.4556
[3] train/valid loss: 2.7434/2.5175 prec: 0.4777
[4] train/valid loss: 2.4140/2.3110 prec: 0.5005
[5] train/valid loss: 2.2685/2.1476 prec: 0.5253
[6] train/valid loss: 2.1209/2.0084 prec: 0.5356
[7] train/valid loss: 1.8629/1.6100 prec: 0.5780
[8] train/valid loss: 1.4851/1.3500 prec: 0.6137
[9] train/valid loss: 1.3214/1.2441 prec: 0.6287
